In [ ]:
from deap import creator, base, tools, algorithms

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [ ]:
toolbox = base.Toolbox()
import random

toolbox.register("random_char", random.choice, "ABCDEFGHIJKLMNOPQRSTUVWXYZ")

DIM = 10 #matrix side

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.random_char, n=DIM * DIM)

In [ ]:
print(toolbox.individual())

In [ ]:
def __str__(individual):
    s = ""
    for i in range(len(individual)):
        s += individual[i]
        if i % DIM == DIM-1:
            s+='#'
    return s

creator.Individual.__str__ = __str__

In [ ]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
#regex list:
hours = ("ONE", "TWO", "THREE", "FOUR", "FIVE", "SIX", "SEVEN", "EIGHT", "NINE", "TEN", "ELEVEN", "TWELVE",)

restrings = []
for h in hours: restrings.append(h+".+O.+CLOCK")
for h in hours: restrings.append("HALF.+PAST.+"+h)
for h in hours: restrings.append("QUARTER.+PAST.+"+h)
for h in hours: restrings.append("QUARTER.+TO.+"+h)

In [ ]:
def evaluateInd(individual):
    import re
    s = str(individual)
    scores = [re.compile(r).search(s) != None for r in restrings]
    return (float(sum(scores)),)

In [ ]:
ind = toolbox.individual()
evaluateInd(ind)

In [ ]:
#build a keyword list for mutation
keywords = set()
for r in restrings:
    for i in r.split(".+"):
        keywords.add(i)
keywords = list(keywords)

In [ ]:
def myMutation(individual):
    kw = random.choice(keywords)
    pos = random.randint(1,len(individual)-len(kw))
    for i, ch in enumerate(kw):
        individual[pos+i]=ch
    return (individual,)

In [ ]:
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("select", tools.selBest)
toolbox.register("evaluate", evaluateInd)
toolbox.register("mutate", myMutation)

In [ ]:
if __name__ == "__main__":
    pop = toolbox.population(n=1000)

    fit = 0.0
    while (fit < len(restrings)):
        
        # --------.-------------- (---, -------, parents, children, probabilities, iterations)
        algorithms.eaMuPlusLambda (pop, toolbox, 400, 100, 0.2, 0.4, 1)

        top = sorted(pop, key=lambda x:x.fitness.values[0])[-1]
        fit = top.fitness.values[0]
        print(fit)

    print(top)

In [ ]:
## DEAP.readthedocs.io

In [ ]:
# OVERVIEW

In [1]:
from deap import base, creator
# Crear clase FITNESS: ( Nombre de la clase, Delegando la clase como fitness, Posibles valores min y max del fitness )
creator.create("FitnessMin", base.Fitness, weights=(-1.0,1.0))

# Crear clase CROMOSOMA: ( Nombre de la clase, type de los valores de la clase, Establecer fitness con la clase anterior )
creator.create("Individual", list, fitness=creator.FitnessMin)

In [2]:
import random
from deap import tools

IND_SIZE = 10

# Functions to initialize populations from individuals

toolbox = base.Toolbox()

# Crear el ATRIBUTO de cromosomas ( Nombre del registro para el toolbox, Función para crear un cromosoma aleatorio )
toolbox.register("attribute", random.random)

# Crear el CROMOSOMA ( Nombre del registro, , La clase cromosoma, La clase atributo, Tamaño de la población )
# Cuando se llame a toolbox.individual(), este llamará a tool.initRepeat()
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=IND_SIZE)

# Crear la POBLACIÓN ( Nombre del registro, , Type del cromosoma, El cromosoma )
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [3]:
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)

In [4]:
toolbox.individual()

[2, 6, 5, 4, 7, 0, 9, 1, 3, 8]

In [5]:
def evaluate(individual):
    return sum(individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [6]:
ind = toolbox.individual()
evaluate(ind)

(45,)

In [9]:
def main():
    pop = toolbox.population(n=50)
    CXPB, MUTPB, NGEN = 0.5, 0.2, 40

    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop